In [39]:
import pandas as pd 
import numpy as np
from pathlib import Path
import re
import yaml 
from yaml import SafeLoader

In [40]:
data_dir = Path('/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data')

original_video_dir = data_dir / 'multicam_video_2024-07-22T10_19_22_cropped_20250325101012'
tiled_videos_dir = original_video_dir / "tiled"
permutation_dirs = {
    f"{f.name}": f for n, f in enumerate(original_video_dir.iterdir()) if "permutation" in f.name
}

lp_dir = data_dir / 'lp'
lp_vid_dir = lp_dir / "dlc10v/labeled-data" / original_video_dir.name 

In [41]:
csv_path = lp_vid_dir / "CollectedData.csv"
data = pd.read_csv(csv_path, header=[0, 1, 2])
with open(lp_dir / 'dlc10v/model_config_dlc10v.yaml') as stream:
    yaml_data = list(yaml.load_all(stream, Loader=SafeLoader))
keypoints = yaml_data[0]['data']['keypoints']
config_p = {key: value / "transform_config.yaml" for key, value in permutation_dirs.items()}
original_views = ['mirror_bottom', 'mirror_left', 'mirror_right', 'mirror_top']

In [42]:
# we want to permute the keypoints columns as names: 

def permute_view_values_by_keypoint(
    df: pd.DataFrame,
    original_views: list[str],
    permutation: list[int]
) -> pd.DataFrame:
    assert len(original_views) == len(permutation), "Permutation and view list must match in length"

    # Get all keypoints by stripping the view suffix
    all_bodyparts = df.columns.get_level_values(1).unique()
    keypoints = sorted(set(
        bp.replace(f"_{view}", "")
        for bp in all_bodyparts
        for view in original_views
        if bp.endswith(f"_{view}")
    ))

    # Create backup of values before overwriting
    original_values = {}
    for keypoint in keypoints:
        for i, view in enumerate(original_views):
            full_bp = f"{keypoint}_{view}"
            for coord in ["x", "y", "likelihood"]:
                col = df.columns[
                    (df.columns.get_level_values(1) == full_bp) &
                    (df.columns.get_level_values(2) == coord)
                ][0]
                original_values[(keypoint, view, coord)] = df[col].copy()

    # Apply permutation: assign to view[i] the values from view[permutation[i]]
    for i, target_view in enumerate(original_views):
        source_view = original_views[permutation[i]]
        for keypoint in keypoints:
            for coord in ["x", "y", "likelihood"]:
                col = df.columns[
                    (df.columns.get_level_values(1) == f"{keypoint}_{target_view}") &
                    (df.columns.get_level_values(2) == coord)
                ][0]
                df[col] = original_values[(keypoint, source_view, coord)]

    return df

In [43]:
def preview_permutation_change(df_before: pd.DataFrame, df_after: pd.DataFrame, keypoint: str, view_from: str, view_to: str):
    for coord in ["x", "y", "likelihood"]:
        col_from = df_before.columns[
            (df_before.columns.get_level_values(1) == f"{keypoint}_{view_from}") &
            (df_before.columns.get_level_values(2) == coord)
        ][0]
        col_to = df_after.columns[
            (df_after.columns.get_level_values(1) == f"{keypoint}_{view_to}") &
            (df_after.columns.get_level_values(2) == coord)
        ][0]

        print(f"\n[{coord}] {keypoint}_{view_to} (should be copied from {view_from}):")
        print("Before:", df_before[col_to].head(5).to_list())
        print("After: ", df_after[col_to].head(5).to_list())
        print("Source:", df_before[col_from].head(5).to_list())


In [47]:
def rotate_central_view(df: pd.DataFrame, view_name: str, command: str, width: int, height: int) -> pd.DataFrame:
    """
    Applies rotation transform to all x/y values of the specified view (e.g., 'mirror_central') in a MultiIndex DataFrame.

    Parameters:
    - df: pandas DataFrame with MultiIndex columns (scorer, bodypart, coord)
    - view_name: str, view to apply transform to (e.g., 'mirror_central')
    - command: str, rotation command (e.g., 'transpose=1', 'vflip,hflip', etc.)
    - width, height: original frame dimensions
    """
    for col in df.columns:
        scorer, bodypart, coord = col
        if view_name in bodypart and coord in ['x', 'y']:
            x_col = df.columns[(df.columns.get_level_values(1) == bodypart) & 
                               (df.columns.get_level_values(2) == 'x')][0]
            y_col = df.columns[(df.columns.get_level_values(1) == bodypart) & 
                               (df.columns.get_level_values(2) == 'y')][0]

            x = df[x_col]
            y = df[y_col]

            if command == "transpose=1":
                new_x = height - y
                new_y = x
            elif command == "vflip,hflip":
                new_x = width - x
                new_y = height - y
            elif command == "transpose=2":
                new_x = y
                new_y = width - x
            else:
                raise ValueError(f"Unsupported command: {command}")

            df[x_col] = new_x
            df[y_col] = new_y

    return df

In [44]:
copy_df = data.copy(1)
new_df = permute_view_values_by_keypoint(data, original_views, [2, 0, 3, 1])